In [71]:
import pandas as pd
import numpy as np
from matplotlib as plt
from pprint import pprint
import pickle
%matplotlib inline

In [2]:
# movie_data = pickle.load( open( "movie_data.pkl", "rb" ) )

maybe need to through out movies that are currently in theatres, because this data wont be accurate for those data points

# Parsing scraped data into Dataframe

In [403]:
movie_keys = ['title','domestic_total', 'runtime','rating', 'release_date' ]
weekly_keys = ['rank','num_theaters','weekly_gross', 'week_num']

In [404]:
def parse_value(num):
    try:
        num = int(num.replace('$', '').replace(',', ''))
    except:
        num = np.nan
    return num
    

In [405]:
def parse_weekly_data(weekly_dict):
    '''a function that creates the attributes that are a function 
       of the weekly data (e.g., drop_off_rate) and others. Minimum case yields
       the number of weeks in theatres.
    '''
    weeks = {}
    weekly_keys = ['rank','num_theaters','weekly_gross', 'week_num']
    for key in weekly_keys:
        weeks[key] = [parse_value(x) for x in weekly_dict[key]]
    return weeks

In [406]:
def parse_movie_data(movie_data):
    ''' Takes a dict of a movie scraped from Box Office Mojo and generates
        a row of for each feature to be added to the dataframe.
    '''
    mvp_movie_data = []
    for movie_dict in movie_data:
        # Only keep movies with correct initial values.
        if '1' == movie_dict['weekly_data']['week_num'][0]:
            movie = [movie_dict[key] for key in movie_keys]
            movie.append(max([int(x) for x in movie_dict['weekly_data']['week_num']]))
            movie.append(parse_weekly_data(movie_dict['weekly_data']))
            mvp_movie_data.append(movie) 
    return pd.DataFrame(mvp_movie_data)

In [671]:
df = parse_movie_data(movie_data)

Subsetted the data set to only include movies with data that started at week 1!

In [672]:
names = movie_keys + ['num_weeks','weekly_data']
df.columns = names

In [673]:
df = df[df['num_weeks']>=2]

Subset set df so that every movie ran for at least 2 weeks, otherwise, there would be nothing to predict for movies that ended in the first week. 

# Processing and transofrming the data for regression

need to create a place holder matrix that has enough columns for all the additional data. so if we consider including 12 weeks of potential training data, for three features, needs to be 3*12 X len(df)

In [674]:
weekly_names =  df.loc[0].weekly_data.keys()[:3]
new_feature_names = [col+ str(i) for col in weekly_names for i in range(1,13)]

In [676]:
full_feature_set = []
for movie in np.array(df):
    weekly_features = [(x + [None]*12)[:12] for x in movie[6].values()[:3]]
    feature = list(itertools.chain(*weekly_features))
    full_feature_set.append(feature)
df_features = pd.DataFrame(full_feature_set,columns = new_feature_`names)


In [687]:
features = pd.concat([df.reset_index(),df_features],axis=1)

# Need to build a smarter way to deal with missing values

In [787]:
with open('full_regression_data.pkl', 'wb') as output:
    pickle.dump(features, output, pickle.HIGHEST_PROTOCOL)